In [4]:
from typing import Tuple
import pandas as pd
from tqdm.notebook import tqdm
import sys
sys.path.append('..')
from evaluation.generated_dataset import GeneratedDataset, load_all_from_config
from evaluation.novelty import NoveltyFilter, filter_by_unique_structure

In [5]:
raw_datasets = {
    "WyFormer": ("WyckoffTransformer",),
#    "DiffCSP": ("DiffCSP",)
}

In [6]:
all_datasets = load_all_from_config(
    datasets=list(raw_datasets.values()) + \
        [("split", "train"), ("split", "val"), ("split", "test")],
    dataset_name="carbon_24")

In [7]:
novelty_reference = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
novelty_filter = NoveltyFilter(novelty_reference)

In [8]:
import evaluation.statistical_evaluator
test_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(all_datasets[('split', 'test')].data)

In [9]:
import evaluation.novelty
train_w_template_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1))
train_strict_AFLOW_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_strict_AFLOW_fingerprint, axis=1))
train_relaxed_AFLOW_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_relaxed_AFLOW_fingerprint, axis=1))

In [10]:
carbon_24 = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data,
    all_datasets[('split', 'test')].data], axis=0, verify_integrity=True)

In [11]:
carbon_24.smact_validity.mean()

1.0

In [12]:
(carbon_24.spacegroup_number == 1).any()

True

In [13]:
carbon_24.spacegroup_number.value_counts()

spacegroup_number
12     3253
2      1573
1      1028
8       532
166     426
63      424
227     355
65      310
10      253
6       215
5       188
69      178
15      147
74      147
51      140
11      135
194     127
44      102
71       87
25       87
38       84
178      56
179      53
139      42
42       29
35       29
14       20
20       17
24       14
229      12
3        11
13       10
191      10
47        8
31        8
64        7
4         5
119       5
9         4
164       3
17        2
140       2
59        2
46        2
136       1
82        1
138       1
72        1
39        1
55        1
70        1
148       1
122       1
36        1
7         1
Name: count, dtype: int64

In [14]:
table = pd.DataFrame(
    index=raw_datasets.keys(), columns=[
        "Novelty (%)", "Structural", "Compositional", 
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements",
        "Novel Template (%)", 
        "Novel AFLOW on P1",
        "Novel AFLOW on !P1",
        "Novel AFLOW (%)",
        "Novel AFLOW relaxed (%)",
        "P1 (%)",
        "Space Group"])
table.index.name = "Method"
E_hull_threshold = 0.08
for name, transformations in tqdm(raw_datasets.items()):
    dataset = all_datasets[transformations]
    unique = filter_by_unique_structure(dataset.data)
    print(f"Unique: {len(unique)} / {len(dataset.data)} = {len(unique) / len(dataset.data)}")
    novel_template = ~unique.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1).isin(train_w_template_set)
    table.loc[name, "Novel Template (%)"] = 100 * novel_template.mean()
    novel_aflow_strict = ~unique.apply(evaluation.novelty.record_to_strict_AFLOW_fingerprint, axis=1).isin(train_strict_AFLOW_set)
    table.loc[name, "Novel AFLOW (%)"] = 100 * novel_aflow_strict.mean()
    novel_aflow_relaxed = ~unique.apply(evaluation.novelty.record_to_relaxed_AFLOW_fingerprint, axis=1).isin(train_relaxed_AFLOW_set)
    table.loc[name, "Novel AFLOW relaxed (%)"] = 100 * novel_aflow_relaxed.mean()
    unique_is_P1 = unique.group == 1
    table.loc[name, "Novel AFLOW on P1"] = 100 * novel_aflow_strict[unique_is_P1].mean()
    table.loc[name, "Novel AFLOW on !P1"] = 100 * novel_aflow_strict[~unique_is_P1].mean()
    if transformations == ("split", "train"):
        novel = unique
    else:
        novel = novelty_filter.get_novel(unique)
    table.loc[name, "Novelty (%)"] = 100 * len(novel) / len(unique)
    if "structural_validity" in novel.columns:
        table.loc[name, "Structural"] = 100 * novel.structural_validity.mean()
        table.loc[name, "Compositional"] = 100 * novel.smact_validity.mean()
    if "cdvae_crystal" in novel.columns:
        cov_metrics = test_evaluator.get_coverage(novel.cdvae_crystal)    
        table.loc[name, "Recall"] = 100 * cov_metrics["cov_recall"]
        table.loc[name, "Precision"] = 100 * cov_metrics["cov_precision"]
        novel = novel[novel.structural_validity]
        table.loc[name, r"$\rho$"] = test_evaluator.get_density_emd(novel)
        table.loc[name, "$E$"] = test_evaluator.get_cdvae_e_emd(novel)
        table.loc[name, "# Elements"] = test_evaluator.get_num_elements_emd(novel)
    table.loc[name, "P1 (%)"] = 100 * (novel.group == 1).mean()
    table.loc[name, "Space Group"] = test_evaluator.get_sg_chi2(novel)
table

  0%|          | 0/1 [00:00<?, ?it/s]

Unique: 522 / 10000 = 0.0522


,Novelty (%),Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements,Novel Template (%),Novel AFLOW on P1,Novel AFLOW on !P1,Novel AFLOW (%),Novel AFLOW relaxed (%),P1 (%),Space Group
Method,,,,,,,,,,,,,,,
WyFormer,53.065134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.065134,54.545455,53.0,53.065134,53.065134,4.33213,4.032625


In [19]:
cdvae_table = pd.DataFrame(index=pd.Index(raw_datasets.keys(), tupleize_cols=False),
    columns=[
        "Structural", "Compositional",
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements"])
sample_size = 992
for name, transformations in tqdm(raw_datasets.items()):
    dataset = all_datasets[transformations]
    if "structure" in dataset.data.columns:
        cdvae_table.loc[name, "Compositional"] = 100*dataset.data.smact_validity.mean()
        cdvae_table.loc[name, "Structural"] = 100*dataset.data.structural_validity.mean()
        valid = dataset.data[dataset.data.naive_validity]
        cov_metrics = test_evaluator.get_coverage(valid.cdvae_crystal)
        cdvae_table.loc[name, "Recall"] = 100*cov_metrics["cov_recall"]
        cdvae_table.loc[name, "Precision"] = 100*cov_metrics["cov_precision"]
        cdvae_table.loc[name, r"$\rho$"] = test_evaluator.get_density_emd(valid)
        cdvae_table.loc[name, "$E$"] = test_evaluator.get_cdvae_e_emd(valid)
        cdvae_table.loc[name, "# Elements"] = test_evaluator.get_num_elements_emd(valid)
cdvae_table.to_csv("tables/cdvae_metrics_no_relax_table.csv")
#prettify(cdvae_table).to_latex("tables/cdvae_metrics_no_relax_table.tex", siunitx=True, convert_css=True)
#prettify(cdvae_table)

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
cdvae_table

,Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements
WyFormer,NaN,NaN,NaN,NaN,NaN,NaN,NaN
